# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation. You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.

In [145]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

### Hyperparameters
Tune the following parameters:

Set epochs to the number of epochs. Set batch_size to the batch size. Set rnn_size to the size of the RNNs. Set num_layers to the number of layers. Set encoding_embedding_size to the size of the embedding for the encoder. Set decoding_embedding_size to the size of the embedding for the decoder. Set learning_rate to the learning rate. Set keep_probability to the Dropout keep probability

In [146]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 20000  # Number of samples to train on.

### Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [147]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en.txt'
target_path = 'data/small_vocab_fr.txt'
#input_text = helper.load_data(source_path)
#target_text = helper.load_data(target_path)

input_characters = set()
target_characters = set()

### Implement Preprocessing Function
Text to Word Ids As you did with other RNNs, you must turn the text into a number so the computer can understand it.

In [148]:
input_texts = []
target_texts = []
with open(source_path, 'r', encoding='utf-8') as f:
    input_lines = f.read().split('\n')
for input_line in input_lines[: min(num_samples, len(input_lines) - 1)]:
    input_text = input_line.split()
    input_texts.append(input_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
with open(target_path, 'r', encoding='utf-8') as f:
    target_lines = f.read().split('\n')
for target_line in target_lines[: min(num_samples, len(target_lines) - 1)]:
    target_text = target_line.split()
    target_texts.append(target_text)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [149]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [150]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 20000
Number of unique input tokens: 227
Number of unique output tokens: 336
Max sequence length for inputs: 17
Max sequence length for outputs: 21


### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [151]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import keras

# Check Keras Version
assert LooseVersion(keras.__version__) >= LooseVersion('1.0'), 'Please use Keras version 2.0.2 or newer'
print('Keras Version: {}'.format(keras.__version__))

Keras Version: 2.1.3


In [152]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [153]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [154]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [155]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [156]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [157]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [158]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Train on 16000 samples, validate on 4000 samples
Epoch 1/100
16000/16000 [==============================] - 16s 973us/step - loss: 1.8342 - val_loss: 1.2530
Epoch 2/100
16000/16000 [==============================] - 13s 803us/step - loss: 0.9177 - val_loss: 0.7305
Epoch 3/100
16000/16000 [==============================] - 13s 803us/step - loss: 0.6471 - val_loss: 0.5761
Epoch 4/100
16000/16000 [==============================] - 13s 801us/step - loss: 0.5481 - val_loss: 0.5218
Epoch 5/100
16000/16000 [==============================] - 13s 800us/step - loss: 0.4812 - val_loss: 0.4516
Epoch 6/100
16000/16000 [==============================] - 13s 806us/step - loss: 0.4124 - val_loss: 0.3768
Epoch 7/100
16000/16000 [==============================] - 13s 806us/step - loss: 0.3384 - val_loss: 0.2994
Epoch 8/100
16000/16000 [==============================] - 13s 800us/step - loss: 0.2666 - val_loss: 0.2588
Epoch 9/100
16000/16000 [==============================] - 13s 803us/step - loss: 0.191

In [159]:
# Save model
model.save('s2s.h5')

C:\Users\ashwa\Anaconda3\envs\tensorflow\lib\site-packages\keras\engine\topology.py:2361: UserWarning: Layer lstm_8 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_7/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_7/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


### Decoding - Inference
Create inference logits using tf.contrib.seq2seq.simple_decoder_fn_inference() and tf.contrib.seq2seq.dynamic_rnn_decoder().

In [160]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

### Build the Decoding Layer
Implement decoding_layer() to create a Decoder RNN layer.

Create RNN cell for decoding using rnn_size and num_layers. Create the output fuction using lambda to transform it's input, logits, to class logits. Use the your decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope, output_fn, keep_prob) function to get the training logits. Use your decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, maximum_length, vocab_size, decoding_scope, output_fn, keep_prob) function to get the inference logits. Note: You'll need to use tf.variable_scope to share variables between training and inference.

In [161]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    print(len(input_seq[0]))
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    #target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char + " "

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > 60):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

### Translate
This will translate translate_sentence from English to French.

In [162]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    #seq_index = 2
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    print(np.array(input_seq[0]).shape)
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

(17, 227)
17
-
Input sentence: ['new', 'jersey', 'is', 'sometimes', 'quiet', 'during', 'autumn', ',', 'and', 'it', 'is', 'snowy', 'in', 'april', '.']
Decoded sentence: est parfois calme à l'automne , et il est neigeux en avril . 
(17, 227)
17
-
Input sentence: ['the', 'united', 'states', 'is', 'usually', 'chilly', 'during', 'july', ',', 'and', 'it', 'is', 'usually', 'freezing', 'in', 'november', '.']
Decoded sentence: est généralement froid en juillet , et il gèle habituellement 
(17, 227)
17
-
Input sentence: ['california', 'is', 'usually', 'quiet', 'during', 'march', ',', 'and', 'it', 'is', 'usually', 'hot', 'in', 'june', '.']
Decoded sentence: généralement généralement calme en mars , et il est généralement 
(17, 227)
17
-
Input sentence: ['the', 'united', 'states', 'is', 'sometimes', 'mild', 'during', 'june', ',', 'and', 'it', 'is', 'cold', 'in', 'september', '.']
Decoded sentence: est parfois légère en juin , et il fait froid en septembre . 
(17, 227)
17
-
Input sentence: ['your',

17
-
Input sentence: ['she', 'dislikes', 'that', 'little', 'red', 'truck', '.']
Decoded sentence: détestait ce petit camion rouge . . . . . . . . . . . . . . . 
(17, 227)
17
-
Input sentence: ['the', 'grapefruit', 'is', 'my', 'most', 'loved', 'fruit', ',', 'but', 'the', 'banana', 'is', 'her', 'most', 'loved', '.']
Decoded sentence: le pamplemousse est mon fruit le plus cher , mais la banane est 
(17, 227)
17
-
Input sentence: ['france', 'is', 'snowy', 'during', 'may', ',', 'and', 'it', 'is', 'never', 'busy', 'in', 'autumn', '.']
Decoded sentence: est neigeux au mois de mai , et il est jamais trop de monde à 
(17, 227)
17
-
Input sentence: ['china', 'is', 'usually', 'mild', 'during', 'winter', ',', 'but', 'it', 'is', 'never', 'busy', 'in', 'february', '.']
Decoded sentence: est généralement calme pendant l' hiver , mais il est jamais 
(17, 227)
17
-
Input sentence: ['china', 'is', 'never', 'nice', 'during', 'july', ',', 'but', 'it', 'is', 'usually', 'snowy', 'in', 'spring', '.']
Decoded

17
-
Input sentence: ['california', 'is', 'never', 'wet', 'during', 'november', ',', 'and', 'it', 'is', 'sometimes', 'pleasant', 'in', 'september', '.']
Decoded sentence: est jamais humide en novembre , et il est parfois agréable en 
(17, 227)
17
-
Input sentence: ['they', 'like', 'lemons', ',', 'limes', ',', 'and', 'grapefruit', '.']
Decoded sentence: aime les citrons , citrons verts et le pamplemousse . . . . . 
(17, 227)
17
-
Input sentence: ['the', 'united', 'states', 'is', 'never', 'beautiful', 'during', 'march', ',', 'and', 'it', 'is', 'usually', 'relaxing', 'in', 'summer', '.']
Decoded sentence: est jamais belle en mars , et il est relaxant habituellement 
(17, 227)
17
-
Input sentence: ['elephants', 'were', 'his', 'most', 'feared', 'animals', '.']
Decoded sentence: étaient ses animaux les plus redoutés . . . . . l' anglais . 
(17, 227)
17
-
Input sentence: ['the', 'strawberry', 'is', 'their', 'least', 'favorite', 'fruit', ',', 'but', 'the', 'apple', 'is', 'our', 'least', 'favor